In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [2]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [3]:
features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "cd_risc1", "active_coping1", "planning1", "positive_reframing1",
               "acceptance1", "humor1", "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
            "denial1",
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "trauma_history8_1"]

In [4]:
bad_features = ["T1bias", "state1", "trait1", "phq1", "PCL1",
                "denial1", "substance_use1", "self_blame1", "trauma_history8_1"]
numerical_features = [ "T1Acc1t", "T1Acc1n", "T1bias", "state1", "trait1", "lot1", "phq1",
                      "cd_risc1", "PCL1", "bad_features", "trauma_history8_1"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", 
                        "lot1", "trait1", "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "active_coping1",
                        "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1", 
                        "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
                        "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1"]

df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

pca = PCA(n_components = 1)
df["pcls"] = pca.fit_transform(df[["PCL1",  "PCL_Broad1", "PCL_Strict1"]])
df.drop(["PCL1",  "PCL_Broad1", "PCL_Strict1"], axis=1, inplace=True)
features.append("pcls")
features.remove("PCL1")
features.remove("PCL_Broad1")
features.remove("PCL_Strict1")


X = df[features]


ss = StandardScaler()
X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [21]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

#eval
pres=[]
for train, test in kfold.split(X_train_2, y_train_2):

    rfe =  RFE(RandomForestClassifier(n_estimators=100), 20).fit(X_train_2[train], y_train_2[train])

    X_train_rfe = rfe.transform(X_train_2[train])
    X_test_rfe = rfe.transform(X_train_2[test])
    
    sm = SMOTE(random_state=27)
    X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train_2[train].ravel())

    # create model
    n_cols = X_train_res.shape[1]
    model = Sequential()


    model.add(Dense(2, activation='elu', input_dim = n_cols))
    model.add(Dropout(0.5))

    model.add(Dense(2, activation='elu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(2, activation='elu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy')


    # Fit the model
    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
        
        # Fit the model
    model.fit(X_train_res, y_train_res, epochs = 500, verbose=0, 
                  validation_split = .1, class_weight = {0:7, 1:1}, callbacks=callbacks)
    
    # evaluate the model                    
    y_pred =  model.predict(X_test_rfe)
    y_pred = y_pred > 0.5

    s = precision_score(y_train_2[test], y_pred)
    print("\n\nprecision\n", s)
    pres.append(s)
    print("recall\n", recall_score(y_train_2[test], y_pred))
print("\n\nmean precision", sum(pres)/len(pres))

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)




precision
 0.0
recall
 0.0


precision
 0.10526315789473684
recall
 0.5714285714285714


precision
 0.03225806451612903
recall
 0.14285714285714285


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)




precision
 0.0
recall
 0.0


precision
 0.07407407407407407
recall
 0.2857142857142857


mean precision 0.04231905929698799


In [ ]:

rfe =  RFE(RandomForestClassifier(n_estimators=100), 5).fit(X_train_2, y_train_2)

X_train_rfe = rfe.transform(X_train_2)
X_test_rfe = rfe.transform(X_test_2)
    
sm = SMOTE(random_state=27)
X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train_2.ravel())

# create model
n_cols = X_train_res.shape[1]
model = Sequential()
    
model.add(Dense(7, activation='elu', input_dim = n_cols))
model.add(Dropout(0.5))

model.add(Dense(5, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')


# Fit the model
model.fit(X_train_res, y_train_res, epochs = 250, verbose=0, class_weight = {0:1, 1:1.7})
    
# evaluate the model                    
y_pred =  model.predict(X_test_rfe)
y_pred = y_pred > 0.5

s = precision_score(y_pred,y_test_2)
print("\n\nprecision\n", s)
print("recall\n", recall_score(y_pred,y_test_2))


In [22]:

rfe =  RFE(RandomForestClassifier(n_estimators=100), 5).fit(X_train, y_train)

X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)
    
sm = SMOTE(random_state=27)
X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train.ravel())

# create model
n_cols = X_train_res.shape[1]
model = Sequential()
    
model.add(Dense(7, activation='elu', input_dim = n_cols))
model.add(Dropout(0.5))

model.add(Dense(5, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')


# Fit the model
model.fit(X_train_res, y_train_res, epochs = 250, verbose=0, class_weight = {0:1, 1:1.7})
    
# evaluate the model                    
y_pred =  model.predict(X_test_rfe)
y_pred = y_pred > 0.5

s = precision_score(y_pred,y_test)
print("\n\nprecision\n", s)
print("recall\n", recall_score(y_pred,y_test))




precision
 1.0
recall
 0.1111111111111111
